# The Connection Between ...

* Yarin Cohen, ID: 211361720
* Amit Shiber, ID: 

## About Our Project

dd

## Step 1: Defining a Research Question

## Step 2: Data Acquisition

### Data Acquisition by Crawling

### Data Acquisition by API

## Step 3: Data Handling

### Missing Data

### Data Duplication

### Outliers

## Step 4: Machine Learning